In [1]:
# import relevant libraries
import numpy as np
import pandas as pd
import seaborn as sns
import time # to measure how long the models take
from sklearn import datasets
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, BaggingClassifier, VotingClassifier
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, precision_recall_curve
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder

In [2]:
train = pd.read_csv(r"/Users/frieda/Desktop/schulich/data/archive/train.csv",sep=";")

In [3]:
test = pd.read_csv(r"/Users/frieda/Desktop/schulich/data/archive/test.csv",sep=";")

In [11]:
train.drop_duplicates(inplace=True)
test.drop_duplicates(inplace=True)


In [12]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45211 entries, 0 to 45210
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   age        45211 non-null  int64 
 1   job        45211 non-null  object
 2   marital    45211 non-null  object
 3   education  45211 non-null  object
 4   default    45211 non-null  object
 5   balance    45211 non-null  int64 
 6   housing    45211 non-null  object
 7   loan       45211 non-null  object
 8   contact    45211 non-null  object
 9   day        45211 non-null  int64 
 10  month      45211 non-null  object
 11  duration   45211 non-null  int64 
 12  campaign   45211 non-null  int64 
 13  pdays      45211 non-null  int64 
 14  previous   45211 non-null  int64 
 15  poutcome   45211 non-null  object
 16  y          45211 non-null  object
dtypes: int64(7), object(10)
memory usage: 6.2+ MB


In [13]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4521 entries, 0 to 4520
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   age        4521 non-null   int64 
 1   job        4521 non-null   object
 2   marital    4521 non-null   object
 3   education  4521 non-null   object
 4   default    4521 non-null   object
 5   balance    4521 non-null   int64 
 6   housing    4521 non-null   object
 7   loan       4521 non-null   object
 8   contact    4521 non-null   object
 9   day        4521 non-null   int64 
 10  month      4521 non-null   object
 11  duration   4521 non-null   int64 
 12  campaign   4521 non-null   int64 
 13  pdays      4521 non-null   int64 
 14  previous   4521 non-null   int64 
 15  poutcome   4521 non-null   object
 16  y          4521 non-null   object
dtypes: int64(7), object(10)
memory usage: 635.8+ KB


In [14]:
train.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


In [15]:
# drop columns
df = train.drop(['day','month','previous','pdays'], axis=1)

In [16]:
# drop columns
df2 = test.drop(['day','month','previous','pdays'], axis=1)

In [17]:
df = pd.get_dummies(df, columns=['y'], drop_first=True)
df2 = pd.get_dummies(df2, columns=['y'], drop_first=True)


In [18]:
df.head()

,age,job,marital,education,default,balance,housing,loan,contact,duration,campaign,poutcome,y_yes
0,58,management,married,tertiary,no,2143,yes,no,unknown,261,1,unknown,0
1,44,technician,single,secondary,no,29,yes,no,unknown,151,1,unknown,0
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,76,1,unknown,0
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,92,1,unknown,0
4,33,unknown,single,unknown,no,1,no,no,unknown,198,1,unknown,0


In [19]:
df2.head()

,age,job,marital,education,default,balance,housing,loan,contact,duration,campaign,poutcome,y_yes
0,30,unemployed,married,primary,no,1787,no,no,cellular,79,1,unknown,0
1,33,services,married,secondary,no,4789,yes,yes,cellular,220,1,failure,0
2,35,management,single,tertiary,no,1350,yes,no,cellular,185,1,failure,0
3,30,management,married,tertiary,no,1476,yes,yes,unknown,199,4,unknown,0
4,59,blue-collar,married,secondary,no,0,yes,no,unknown,226,1,unknown,0


In [32]:
df.describe()

,age,balance,duration,campaign,y_yes
count,45211.000000,45211.000000,45211.000000,45211.000000,45211.000000
mean,40.936210,1362.272058,258.163080,2.763841,0.116985
std,10.618762,3044.765829,257.527812,3.098021,0.321406
min,18.000000,-8019.000000,0.000000,1.000000,0.000000
25%,33.000000,72.000000,103.000000,1.000000,0.000000
50%,39.000000,448.000000,180.000000,2.000000,0.000000
75%,48.000000,1428.000000,319.000000,3.000000,0.000000
max,95.000000,102127.000000,4918.000000,63.000000,1.000000


In [33]:
df2.describe()

,age,balance,duration,campaign,y_yes
count,4521.000000,4521.000000,4521.000000,4521.000000,4521.000000
mean,41.170095,1422.657819,263.961292,2.793630,0.115240
std,10.576211,3009.638142,259.856633,3.109807,0.319347
min,19.000000,-3313.000000,4.000000,1.000000,0.000000
25%,33.000000,69.000000,104.000000,1.000000,0.000000
50%,39.000000,444.000000,185.000000,2.000000,0.000000
75%,49.000000,1480.000000,329.000000,3.000000,0.000000
max,87.000000,71188.000000,3025.000000,50.000000,1.000000


In [31]:

(df['y_yes'] ==1).sum()/(df['y_yes']==0).sum()


0.1324833425179099

In [28]:
(df2['y_yes'] == 1).sum()/(df2['y_yes'] == 0).sum()

0.13025